In [ ]:
# !pip install crewai
# !pip install crewai-tools

In [ ]:
import os
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool

from google.colab import userdata
gemini_key = "YOUR_GEMINI_API_KEY"
serper_key = "YOUR_SERPER_API_KEY"
os.environ["GOOGLE_API_KEY"] = gemini_key
os.environ["SERPER_API_KEY"] = serper_key

# Initialize a search tool (to fetch real-time travel info)
search_tool = SerperDevTool()

# Define the AI Model
llm = LLM(model="gemini/gemini-1.5-flash",
          verbose=True,
          temperature=0.5,
          api_key=os.environ["GOOGLE_API_KEY"])

# Travel Researcher Agent (Finds historical sites + weather)
researcher = Agent(
    role="Travel Researcher",
    goal="Find historical sites, public transport hotels, and real-time weather for {destination}.",
    verbose=True,
    memory=True,
    backstory="You are an expert travel researcher, providing up-to-date information about history-focused trips.",
    llm=llm,
    tools=[search_tool],  # Uses live search tool
    allow_delegation=True
)

# 💰 Budget Planner Agent (Ensures trip stays under $1500)
budget_planner = Agent(
    role="Budget Planner",
    goal="Find budget flights, hotels, and activities within {budget} for {destination}.",
    verbose=True,
    memory=True,
    backstory="You are a skilled budget analyst ensuring trips fit within financial constraints.",
    llm=llm,
    tools=[search_tool],
    allow_delegation=False
)

# 🗺️ Itinerary Planner Agent (Creates a balanced 3-day plan)
itinerary_planner = Agent(
    role="Itinerary Planner",
    goal="Create a 3-day itinerary for {destination}, ensuring all historical sites are covered under {budget}.",
    verbose=True,
    memory=True,
    backstory="You are an expert in trip planning, ensuring travelers get the best experience within their budget.",
    llm=llm,
    tools=[search_tool],
    allow_delegation=False
)

# 📝 Travel Research Task
research_task = Task(
    description="Find the best historical sites, weather forecast, and public transport hotels for {destination}.",
    expected_output="A list of top historical sites, a real-time weather update, and 3 hotel options near public transport.",
    tools=[search_tool],
    agent=researcher
)

# 💲 Budget Estimation Task
budget_task = Task(
    description="Find budget flights, hotel options, and daily food/transport costs for {destination}. Ensure total cost stays under {budget}.",
    expected_output="A full cost breakdown (flights, hotel, food, attractions) ensuring a $1500 budget is maintained.",
    tools=[search_tool],
    agent=budget_planner
)

# 📅 Itinerary Planning Task
itinerary_task = Task(
    description="Plan a 3-day itinerary for {destination}, focusing on historical sites, budget constraints, and real-time weather conditions.",
    expected_output="A detailed 3-day plan, considering weather and budget constraints, with transport recommendations.",
    tools=[search_tool],
    agent=itinerary_planner
)

# 🚀 Crew Setup: All agents working together!
crew = Crew(
    agents=[researcher, budget_planner, itinerary_planner],
    tasks=[research_task, budget_task, itinerary_task],
    process=Process.sequential  # Runs tasks in sequence
)

# 🔥 Run the CrewAI Trip Advisor system for London with a $1500 budget
result = crew.kickoff(inputs={'destination': 'London', 'budget': '1500'})
print(result)

In [4]:
# streamlit App

In [ ]:
!pip install streamlit
!pip -q install pyngrok

In [ ]:
# Set API Keys (Replace with your actual keys)
import os
gemini_key = "YOUR_GEMINI_API_KEY"
serper_key = "YOUR_SERPER_API_KEY"
os.environ["GOOGLE_API_KEY"] = gemini_key
os.environ["SERPER_API_KEY"] = serper_key

In [ ]:
%%writefile app.py

import os
import streamlit as st
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool



# Initialize search tool for real-time travel info
search_tool = SerperDevTool()

# Define AI Model
llm = LLM(model="gemini/gemini-1.5-flash",
          verbose=True,
          temperature=0.5,
          api_key=os.environ["GOOGLE_API_KEY"])

# Define function to create agents
def create_agents(destination, budget):
    researcher = Agent(
        role="Travel Researcher",
        goal=f"Find historical sites, public transport hotels, and real-time weather for {destination}.",
        backstory="You are an expert travel researcher, providing up-to-date information about history-focused trips.",
        verbose=True,
        memory=True,
        llm=llm,
        tools=[search_tool]
    )

    budget_planner = Agent(
        role="Budget Planner",
        goal=f"Find budget flights, hotels, and activities within {budget} for {destination}.",
        backstory="You are a skilled budget analyst ensuring trips fit within financial constraints.",
        verbose=True,
        memory=True,
        llm=llm,
        tools=[search_tool]
    )

    itinerary_planner = Agent(
        role="Itinerary Planner",
        goal=f"Create a 3-day itinerary for {destination}, ensuring all historical sites are covered under {budget}.",
        backstory="You are an expert in trip planning, ensuring travelers get the best experience within their budget.",
        verbose=True,
        memory=True,
        llm=llm,
        tools=[search_tool]
    )
    return researcher, budget_planner, itinerary_planner

# Streamlit UI
st.title("🌍 AI-Powered Travel Planner")

destination = st.text_input("Enter Destination:")
budget = st.text_input("Enter Budget (INR):")

if st.button("Generate Travel Plan"):
    if not destination or not budget:
        st.error("Please enter both a destination and budget.")
    else:
        st.info("🧳 Generating your travel plan...")

        # Create agents
        researcher, budget_planner, itinerary_planner = create_agents()

        # Define tasks
        research_task = Task(
            description=f"Find the best historical sites, weather forecast, and public transport hotels for {destination}.",
            expected_output="A list of top historical sites, a real-time weather update, and 3 hotel options near public transport.",
            agent=researcher
        )

        budget_task = Task(
            description=f"Find budget flights, hotel options, and daily costs for {destination} within {budget}.",
            expected_output=f"A full cost breakdown (flights, hotel, food, attractions) ensuring a {budget} budget is maintained.",
            agent=budget_planner
        )

        itinerary_task = Task(
            description=f"Plan a 3-day itinerary for {destination} under {budget}.",
            expected_output="A detailed 3-day plan, considering weather and budget constraints, with transport recommendations.",
            agent=itinerary_planner
        )

        # Create Crew
        crew = Crew(
            agents=[researcher, budget_planner, itinerary_planner],
            tasks=[research_task, budget_task, itinerary_task],
            process=Process.sequential
        )

        # Run AI agents
        responses = crew.kickoff(inputs={'destination': destination, 'budget': budget})

         # Display each agent's output separately
        st.subheader("📌 Travel Researcher Findings")
        st.write(responses[0])

        st.subheader("💰 Budget Planner Suggestions")
        st.write(responses[1])

        st.subheader("🗺️ Itinerary Planner Recommendations")
        st.write(responses[2])

        # Display results
        st.success("✅ Travel Plan Generated!")
        st.subheader("Your Travel Plan:")
        st.write(responses)


In [ ]:
%%writefile app.py
import os
import streamlit as st
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool

# --- Initialize search tool ---
search_tool = SerperDevTool()

# --- Define AI Model ---
llm = LLM(model="gemini/gemini-1.5-flash",
          verbose=True,
          temperature=0.5,
          api_key=os.environ.get("GOOGLE_API_KEY", ""))

# --- Streamlit UI ---
st.title("🌍 AI-Powered Travel Planner")
st.markdown("**Plan your perfect trip with AI-powered insights!**")

destination = st.text_input("📍 Enter Destination:")
budget = st.text_input("💰 Enter Budget (INR):")

# --- Function to create AI Agents ---
def create_agents(destination, budget):
    researcher = Agent(
        role="Travel Researcher",
        goal=f"Find historical sites, public transport hotels, and real-time weather for {destination}.",
        backstory="You are an expert travel researcher, providing up-to-date information about history-focused trips.",
        verbose=True,
        memory=True,
        llm=llm,
        tools=[search_tool]
    )

    budget_planner = Agent(
        role="Budget Planner",
        goal=f"Find budget flights, hotels, and activities within {budget} for {destination}.",
        backstory="You are a skilled budget analyst ensuring trips fit within financial constraints.",
        verbose=True,
        memory=True,
        llm=llm,
        tools=[search_tool]
    )

    itinerary_planner = Agent(
        role="Itinerary Planner",
        goal=f"Create a 3-day itinerary for {destination}, ensuring all historical sites are covered under {budget}.",
        backstory="You are an expert in trip planning, ensuring travelers get the best experience within their budget.",
        verbose=True,
        memory=True,
        llm=llm,
        tools=[search_tool]
    )

    return researcher, budget_planner, itinerary_planner

# --- Generate Travel Plan Button ---
if st.button("🎯 Generate Travel Plan"):
    if not destination or not budget:
        st.error("⚠️ Please enter both a destination and budget.")
    else:
        st.info("⏳ Generating your AI-powered travel plan...")

        # Create AI Agents
        researcher, budget_planner, itinerary_planner = create_agents(destination, budget)

        # Define AI Tasks
        research_task = Task(
            description=f"Find the best historical sites, weather forecast, and public transport hotels for {destination}.",
            expected_output="✅ A list of top historical sites, a real-time weather update, and 3 hotel options near public transport.",
            agent=researcher
        )

        budget_task = Task(
            description=f"Find budget flights, hotel options, and daily costs for {destination} within {budget}.",
            expected_output=f"✅ A full cost breakdown (flights, hotel, food, attractions) ensuring a {budget} budget is maintained.",
            agent=budget_planner
        )

        itinerary_task = Task(
            description=f"Plan a 3-day itinerary for {destination} under {budget}.",
            expected_output="✅ A detailed 3-day plan, considering weather and budget constraints, with transport recommendations.",
            agent=itinerary_planner
        )

        # --- Create Crew ---
        crew = Crew(
            agents=[researcher, budget_planner, itinerary_planner],
            tasks=[research_task, budget_task, itinerary_task],
            process=Process.sequential
        )

        # --- Run AI Agents ---
        responses = crew.kickoff(inputs={'destination': destination, 'budget': budget})

        # --- Display Results ---
        st.success("✅ Travel Plan Generated!")

        # Clean and display the output
        for agent_name, response in zip(["Travel Researcher", "Budget Planner", "Itinerary Planner"], responses):
            st.subheader(f"📌 {agent_name} Findings")

            # Ensure response is properly formatted
            if isinstance(response, tuple) and len(response) > 1:
                clean_response = response[1]  # Extract the actual text from tuple
            else:
                clean_response = response

            st.write(clean_response if clean_response else "No response available.")


st.markdown("🌍 *Enjoy your AI-powered trip planning!* 🚀")


In [ ]:
from pyngrok import ngrok
ngrok.kill()
outh_token='2KVWElPAg1tzrBVnxmFFYaxfeyg_2cCYD9fH3HpMqU1u5TEsC' # we will not provide any token
ngrok.set_auth_token(outh_token)

# create the tunnel
ngrok_tunnel=ngrok.connect(addr="5000",proto="http")
print("Tracking uri:",ngrok_tunnel.public_url)

!streamlit run --server.port 5000 app.py